# dEdx weights calculation

## Input

This notebook expects to find a txt file called "VolumesZPosition.txt". This file is coming from the _SimTracker/TrackerMaterialAnalysis_ package. Just for the record we correct for the Materials with spaces in their name: 

sed -i -e 's/M_NEMA\ FR4\ plate/M_NEMA_FR4_plate/g' VolumesZPosition.txt

but ok now we use HGC_G10-FR4.

In [1]:
inputfile = "VolumesZPosition.txt"

## Some imports

In [2]:
from collections import OrderedDict
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pylab as plt
import seaborn as sns

## Materials Properties

### Materials radiation lengths from Chris, Geant (in mm)

In [3]:
#In mm
#-------- 
FromChrisMatXo = OrderedDict()
FromChrisMatXo['Copper'] = 14.36
FromChrisMatXo['H_Scintillator'] = 425.4
FromChrisMatXo['Cables'] = 0.
#FromChrisMatXo['M_NEMA_FR4_plate'] = 175.
#Just duplicate the above in this case
FromChrisMatXo['HGC_G10-FR4'] = 175.
FromChrisMatXo['Silicon'] = 93.66
FromChrisMatXo['Other'] = 0.
FromChrisMatXo['Air'] = 300000.
FromChrisMatXo['StainlessSteel'] = 17.35
FromChrisMatXo['WCu'] = 5.122
FromChrisMatXo['Lead'] = 5.612
#-------- 
FromGeantMatXo = OrderedDict()
FromGeantMatXo['Copper'] = 14.3559
FromGeantMatXo['H_Scintillator'] = 425.393
FromGeantMatXo['Cables'] = 66.722
#FromGeantMatXo['M_NEMA_FR4_plate'] = 175.056
FromGeantMatXo['HGC_G10-FR4'] = 175.056
FromGeantMatXo['Silicon'] = 93.6762
FromGeantMatXo['Other'] = 0.
FromGeantMatXo['Air'] = 301522.
FromGeantMatXo['StainlessSteel'] = 17.3555
FromGeantMatXo['WCu'] = 5.1225
FromGeantMatXo['Lead'] = 5.6118
#-------- 
df = pd.DataFrame.from_dict([FromChrisMatXo, FromGeantMatXo])
df = df.transpose()
df.columns = ['FromChris', 'FromGeant']
print( '\033[1m' + '            Radiation Lengths in mm')
df

            Radiation Lengths in mm


,FromChris,FromGeant
Copper,14.360,14.3559
H_Scintillator,425.400,425.3930
Cables,0.000,66.7220
HGC_G10-FR4,175.000,175.0560
Silicon,93.660,93.6762
Other,0.000,0.0000
Air,300000.000,301522.0000
StainlessSteel,17.350,17.3555
WCu,5.122,5.1225
Lead,5.612,5.6118


### Materials nuclear interaction lengths from Chris, Geant (in mm)

In [4]:
#In mm
#-------- 
MatNucIntLength = OrderedDict()
FromChrisMatNucIntLength = OrderedDict()
FromChrisMatNucIntLength['Copper'] = 155.1
FromChrisMatNucIntLength['H_Scintillator'] = 701.3
FromChrisMatNucIntLength['Cables'] = 0.
#FromChrisMatNucIntLength['M_NEMA_FR4_plate'] = 484.2
#Just duplicate the above in this case
FromChrisMatNucIntLength['HGC_G10-FR4'] = 484.2
FromChrisMatNucIntLength['Silicon'] = 457.5
FromChrisMatNucIntLength['Other'] = 0.
FromChrisMatNucIntLength['Air'] = 700000
FromChrisMatNucIntLength['StainlessSteel'] = 166
FromChrisMatNucIntLength['WCu'] = 119.9
FromChrisMatNucIntLength['Lead'] = 182.6
#-------- 
FromGeantMatNucIntLength = OrderedDict()
FromGeantMatNucIntLength['Copper'] = 155.88
FromGeantMatNucIntLength['H_Scintillator'] = 700.034
FromGeantMatNucIntLength['Cables'] = 393.71
#FromGeantMatNucIntLength['M_NEMA_FR4_plate'] = 483.429
FromGeantMatNucIntLength['HGC_G10-FR4'] = 483.429
FromGeantMatNucIntLength['Silicon'] = 456.628
FromGeantMatNucIntLength['Other'] = 0.
FromGeantMatNucIntLength['Air'] = 704083
FromGeantMatNucIntLength['StainlessSteel'] = 166.272
FromGeantMatNucIntLength['WCu'] = 120.105
FromGeantMatNucIntLength['Lead'] = 182.472
#-------- 
df = pd.DataFrame.from_dict([FromChrisMatNucIntLength, FromGeantMatNucIntLength])
df = df.transpose()
df.columns = ['FromChris', 'FromGeant']
print( '\033[1m' + '            Nuclear interaction Lengths in mm')
df

            Nuclear interaction Lengths in mm


,FromChris,FromGeant
Copper,155.1,155.880
H_Scintillator,701.3,700.034
Cables,0.0,393.710
HGC_G10-FR4,484.2,483.429
Silicon,457.5,456.628
Other,0.0,0.000
Air,700000.0,704083.000
StainlessSteel,166.0,166.272
WCu,119.9,120.105
Lead,182.6,182.472


### Materials dEdx from PDG, Chris, Geant (in MeV/mm)

In [5]:
#First in MeV gr^-1 cm^2 and at the end will convert to MeV/mm
dEdx = OrderedDict()
#In gr/cm^3 
rho = OrderedDict()
#--------
#Some elements necessary to build our materials
#Note: Whenever in PDG values (2018) are given in more than one state (e.g gas, liquid)
#we go to the materials.xml file and choose the corresponding value. 
dEdx['Fe'] = 1.451
dEdx['Mn'] = 1.428
dEdx['Cr'] = 1.456
dEdx['Ni'] = 1.468
dEdx['C']  = 1.745
dEdx['0']  = 1.801 
dEdx['H']  = 4.034
dEdx['Br'] = 1.380
dEdx['W']  = 1.145
dEdx['N']  = 1.813
dEdx['Al'] = 1.615


#-------- 
rho['Fe'] = 7.874 
rho['Mn'] = 7.440 
rho['Cr'] = 7.180
rho['Ni'] = 8.902 
rho['C']  = 2.265 
rho['0']  = 1.43**-3 #Here we choose the materials.xml value.
rho['H']  = 0.07080
rho['Br'] = 3.103 
rho['W']  = 19.30 
rho['N']  = 0.8070  
rho['Ar'] = 1.639**-3 #Here we choose the materials.xml value. 
rho['Al'] = 2.7

#--------
#Detector materials (From PDG and Geometry/CMSCommonData/data/materials.xml)
#You need a measurement of the density for a compound which we take from the same file
dEdx['Epoxy'] = 0.53539691*dEdx['C'] + 0.13179314*dEdx['H'] + 0.33280996*dEdx['0']
rho['Epoxy'] = 1.3

dEdx['Kapton'] = 0.59985105*dEdx['C'] + 0.080541353*dEdx['H'] + 0.31960759*dEdx['0']
rho['Kapton'] = 1.11

dEdx['Copper'] = 1.403
rho['Copper'] = 8.960

dEdx['H_Scintillator'] = 0.91512109*dEdx['C'] + 0.084878906*dEdx['H']
rho['H_Scintillator'] = 1.032

dEdx['Silicon'] = 1.664
rho['Silicon'] = 2.329

#dEdx['M_NEMA_FR4_plate'] = 0.18077359*dEdx['Silicon'] + 0.4056325*dEdx['0'] + 0.27804208*dEdx['C'] + 0.068442752*dEdx['H'] + 0.067109079*dEdx['Br']
#rho['M_NEMA_FR4_plate'] = 1.025
dEdx['HGC_G10-FR4'] = 0.18077359*dEdx['Silicon'] + 0.4056325*dEdx['0'] + 0.27804208*dEdx['C'] + 0.068442752*dEdx['H'] + 0.067109079*dEdx['Br']
rho['HGC_G10-FR4'] = 1.70


dEdx['Other'] = 0.
rho['Other'] = 0.

#dEdx['Air'] = 0.7494*dEdx['N'] + 0.2369*dEdx['0'] + 0.0129*dEdx['Ar'] + 0.0008*dEdx['H']
dEdx['Air'] = 0.
rho['Air'] = 1.214**-3

dEdx['StainlessSteel'] = 0.6996*dEdx['Fe']+0.01*dEdx['Mn']+0.19*dEdx['Cr']+0.1*dEdx['Ni']+0.0004*dEdx['C'];
rho['StainlessSteel'] = 8.02

dEdx['WCu'] = 0.75*dEdx['W']+0.25*dEdx['Copper']
rho['WCu'] = 14.979

dEdx['Lead'] = 1.122 #Pb
rho['Lead'] = 11.35  #Pb

dEdx['Cables'] = 0.586*dEdx['Copper'] + 0.259*dEdx['C'] + 0.138*dEdx['0'] + 0.017*dEdx['H']
rho['Cables'] = 2.68
#--------
# Now to the calculation. First, we will loop through the composite materials, not 
# to mess with the elements. 
composite_materials = ['Cables','Epoxy','Kapton','StainlessSteel','M_NEMA_FR4_plate','H_Scintillator', 'Air', 'WCu']
for element in dEdx: 
    if element not in composite_materials: continue
    dEdx[element] = (rho[element] * dEdx[element]) / 10.
#And for the rest
for element in dEdx: 
    if element in composite_materials: continue
    dEdx[element] = (rho[element] * dEdx[element]) / 10.


#--------
FromChrisdEdx = OrderedDict()
FromChrisdEdx['Copper'] = 1.26
FromChrisdEdx['H_Scintillator'] = 0.395
FromChrisdEdx['Epoxy'] = 0.
FromChrisdEdx['Kapton'] = 0.
FromChrisdEdx['Cables'] = 0.
FromChrisdEdx['Al'] = 0.
#FromChrisdEdx['M_NEMA_FR4_plate'] = 0.322
#Just duplicate the above
FromChrisdEdx['HGC_G10-FR4'] = 0.322
FromChrisdEdx['Silicon'] = 0.388
FromChrisdEdx['Other'] = 0.
FromChrisdEdx['Air'] = 0.
FromChrisdEdx['StainlessSteel'] = 1.14
FromChrisdEdx['WCu'] = 1.81
FromChrisdEdx['Lead'] = 1.27
#-------- 
#The range below is from commands like: 
# Always recheck columns in txt, so that array is pointing to desired column. 
#array=($(cat VolumesZPosition.txt | grep Copper | awk '{print $7}'))
#IFS=$'\n'
#Highest value
#echo "${array[*]}" | sort -nr | head -n1
#Lowest value
#echo "${array[*]}" | sort -nr | tail -n1

FromGeantdEdxWithGetDEDX = OrderedDict()
FromGeantdEdxWithGetDEDX['Copper'] = "1.14494 - 1.19191"
FromGeantdEdxWithGetDEDX['H_Scintillator'] = "0.155602 - 0.156968"
FromGeantdEdxWithGetDEDX['Cables'] = "0.334163 - 0.343928"
#FromGeantdEdxWithGetDEDX['M_NEMA_FR4_plate'] = "0.200079 - 0.203959"
FromGeantdEdxWithGetDEDX['Epoxy'] = 0.
FromGeantdEdxWithGetDEDX['Kapton'] = 0.
FromGeantdEdxWithGetDEDX['Al'] = 0.
FromGeantdEdxWithGetDEDX['HGC_G10-FR4'] = "0.200079 - 0.203959"
FromGeantdEdxWithGetDEDX['Silicon'] = "0.298729 - 0.349232"
FromGeantdEdxWithGetDEDX['Other'] = 0.
FromGeantdEdxWithGetDEDX['Air'] = "0.000162487 - 0.000200878"
FromGeantdEdxWithGetDEDX['StainlessSteel'] = "1.17868 - 1.24005"
FromGeantdEdxWithGetDEDX['WCu'] = "1.4676 - 1.54044"
FromGeantdEdxWithGetDEDX['Lead'] = "1.28276 - 1.39512"
#-------- 
#The range below is from commands like: 
# Always recheck columns in txt, so that array is pointing to desired column. 
#array=($(cat VolumesZPosition.txt | grep Copper | awk '{print $9}'))
#IFS=$'\n'
#Highest value
#echo "${array[*]}" | sort -nr | head -n1
#Lowest value
#echo "${array[*]}" | sort -nr | tail -n1
FromGeantdEdxWithComputeTotalDEDX = OrderedDict()
FromGeantdEdxWithComputeTotalDEDX['Copper'] = "1.25937 - 1.51496"
FromGeantdEdxWithComputeTotalDEDX['H_Scintillator'] = "0.212843 - 0.230636"
FromGeantdEdxWithComputeTotalDEDX['Cables'] = "0.458855 - 0.507352"
#FromGeantdEdxWithComputeTotalDEDX['M_NEMA_FR4_plate'] = "0.319344 - 0.364818"
FromGeantdEdxWithComputeTotalDEDX['Epoxy'] = 0.
FromGeantdEdxWithComputeTotalDEDX['Kapton'] = 0.
FromGeantdEdxWithComputeTotalDEDX['Al'] = 0.
FromGeantdEdxWithComputeTotalDEDX['HGC_G10-FR4'] = "0.319344 - 0.364818"
FromGeantdEdxWithComputeTotalDEDX['Silicon'] = "0.38849 - 0.442262"
FromGeantdEdxWithComputeTotalDEDX['Other'] = 0.
FromGeantdEdxWithComputeTotalDEDX['Air'] = "0.000221838 - 0.000303024"
FromGeantdEdxWithComputeTotalDEDX['StainlessSteel'] = "1.18145 - 1.40497"
FromGeantdEdxWithComputeTotalDEDX['WCu'] = "1.85741 - 2.18222"
FromGeantdEdxWithComputeTotalDEDX['Lead'] = "1.28388 - 1.52941"
#-------- 
df = pd.DataFrame.from_dict([dEdx,FromChrisdEdx, FromGeantdEdxWithGetDEDX, FromGeantdEdxWithComputeTotalDEDX])
df = df.transpose()[10:]
df.columns = ['FromPDG','FromChris', 'FromGeantWithGetDEDX', 'FromGeantWithComputeTotalDEDX']
print( '\033[1m' + '                              dEdx in MeV/mm')
df

                              dEdx in MeV/mm


,FromPDG,FromChris,FromGeantWithGetDEDX,FromGeantWithComputeTotalDEDX
Al,0.43605,0,0,0
Epoxy,0.268491,0,0,0
Kapton,0.216146,0,0,0
Copper,1.25709,1.26,1.14494 - 1.19191,1.25937 - 1.51496
H_Scintillator,0.200135,0.395,0.155602 - 0.156968,0.212843 - 0.230636
Silicon,0.387546,0.388,0.298729 - 0.349232,0.38849 - 0.442262
HGC_G10-FR4,0.320491,0.322,0.200079 - 0.203959,0.319344 - 0.364818
Other,0,0,0,0
Air,0,0,0.000162487 - 0.000200878,0.000221838 - 0.000303024
StainlessSteel,1.16574,1.14,1.17868 - 1.24005,1.18145 - 1.40497


### Choice for radiation lengths values -> Geant values

In [6]:
# Current choice -> Geant values
#In mm
MatXo = OrderedDict()
MatXo['Copper'] = 14.3559
MatXo['H_Scintillator'] = 425.393
MatXo['Cables'] = 66.722
MatXo['Epoxy'] = 315.901
MatXo['Kapton'] = 365.309
#MatXo['M_NEMA_FR4_plate'] = 175.056
MatXo['HGC_G10-FR4'] = 175.056
MatXo['Silicon'] = 93.6762
MatXo['Other'] = 0.
MatXo['Air'] = 301522.
MatXo['StainlessSteel'] = 17.3555
MatXo['WCu'] = 5.1225
MatXo['Lead'] = 5.6118

In [7]:
# Let's read the input file
# Mat is prepoint material, Z is post point material start, so 
# upper edge of prepoint material. Etable,Efull is the energy loss in the 
# prepoint volume with GetDEDX and ComputeTotalDEDX.  
matZ = pd.read_csv(inputfile, sep=" ", header=None, names=["Mat", "Z", "Eta", "R", "Etable", "Efull"], index_col=False)

In [8]:
# We will make a new column with the physical thickness of the volumes in mm
matZ["PhysThickInmm"] = abs(matZ["Z"].shift(-1) -  matZ["Z"])
matZ["PhysThickInmm"] = matZ["PhysThickInmm"].shift(1)

In [9]:
# We will add a column that indicates the track the relevant volume belongs to. 
# The logic is that right before the next track "PhysThickInmm" column will be 
# very large. 
matZ["trackflag"] = matZ.apply(lambda row: True if row["PhysThickInmm"] < 2000. else False ,axis=1)
matZ["tracknum"] = (( matZ["trackflag"] == False) & (matZ["trackflag"].shift() == True)).cumsum()
matZ["tracknum"] = matZ["tracknum"].shift(1)
matZ = matZ.drop('trackflag', 1)

In [10]:
# Now that we have the tracknum we will not let the last Copper volume to 
# its huge thickness due to track change. According to TDR BH back is at 5137.7 mm 
# so we put 6.0 mm for that Copper. In any case this do *not* effect the 
# dEdx weights calculation since it is after the sensitive material of the last layer. 
# UPDATE: Seems that now last volume is SS. No effect but in any case we put 35/66 mm -> 35
matZ.loc[ matZ["PhysThickInmm"] > 2000. , "PhysThickInmm" ] = 35.0

In [11]:
# The line below is when comparing with Chris which has no Cables. 
# matZ = matZ.query('Mat != "Cables"')

# Again, adding a new column with the physical thickness of the volumes in radiation lengths
matZ["PhysThickInXo"] = matZ.apply(lambda row: row["PhysThickInmm"] / MatXo[row["Mat"]],axis=1)

In [12]:
# Adding a new column with the dEdx of the material that the volumes is build
matZ["dEdx"] = matZ.apply(lambda row: dEdx[row["Mat"]],axis=1)

In [13]:
# Another column with the dEdx times thickness to help us with the calculation of the 
# final dEdx weights 
matZ["dEdxtimesdx"] = matZ["dEdx"] * matZ["PhysThickInmm"]

In [14]:
# And here a column with the cumulative sum
matZ["dEdxtimesdxCum"] = matZ.groupby('tracknum')["dEdxtimesdx"].cumsum()

In [15]:
# And here a column with the cumulative sum for Etable
matZ["EtableCum"] = matZ.groupby('tracknum')["Etable"].cumsum()

In [16]:
# And here a column with the cumulative sum for Efull
matZ["EfullCum"] = matZ.groupby('tracknum')["Efull"].cumsum()

In [17]:
# And here a column for the cumulative sum in radiation length 
matZ["PhysThickInXoCum"] = matZ.groupby('tracknum')["PhysThickInXo"].cumsum()

In [18]:
# We will add a column that indicates the layer the relevant volume belongs to. 
# The logic is that if the previous material is Silicon or Scintillator
# we change layer.
matZ["layerflag"] = matZ.apply(lambda row: False if row["Mat"] == "Silicon" or row["Mat"] == "H_Scintillator" else True ,axis=1)
matZ["layer"] = ( matZ["layerflag"] == True) & (matZ["layerflag"].shift(1) == False) 
matZ["layer"] = matZ.groupby('tracknum')["layer"].cumsum()
#The convention is that layers starts from 1
matZ["layer"] = matZ.apply(lambda row: row["layer"] + 1,axis=1)

In [19]:
# Drop auxillary columns
#We need layerflag for not counting the silicon/scintillator in the dedx. 
#matZ = matZ.drop('layerflag', 1)

In [20]:
# This was my misunderstanding. There should be 53 in the layers column since 
# after the last scintillator or silicon the index will increase, there is 
# material there. In case we want to filter them out uncomment the following. 
# Be careful! Filter chooses and not disgards!
# matZ = matZ.groupby('tracknum').filter(lambda g: ~(g['layer'] == 53.0).any()  ) 

In [23]:
#display(matZ)
#matZ[(matZ["layer"] == 1) & matZ["tracknum"] == 20]
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#matZ[(matZ["tracknum"] == 6.0)]
matZ = matZ.dropna()
#matZ[ matZ['tracknum'] == 2.0]
#matZ[(matZ["Mat"] == "Kapton") & (matZ["tracknum"] == 2.0)]
#matZ[(matZ["Mat"] == "Kapton") & (matZ["layer"] == 34)]
#matZ[(matZ["Mat"] == "Air") & (abs(matZ["eta"]) > 1.5) & (abs(matZ["eta"]) < 2.0)]
#matZ[ (matZ["Z"] > 3950) & (matZ["Z"] < 4000) & (matZ["Mat"] == "Silicon") & (matZ["R"] < 900)]
#matZ[ (matZ["Mat"] == "Silicon") & (matZ["tracknum"] == 5) ]
#matZ[ (matZ["Mat"] == "Silicon") & (matZ["Z"] < 3900) & (matZ["Z"] > 0) & (matZ["PhysThickInmm"] > 0.22)]
#matZ[ (matZ["Mat"] == "Silicon") & (matZ["layer"] > 28) & (matZ["layer"] < 36)& (matZ["PhysThickInmm"] < 0.22)]
matZ[matZ["layer"] >= 28]
#matZ[(matZ["Mat"] == "WCu") & (matZ["Z"] > 3950) & (matZ["Z"] < 4000)]

,Mat,Z,Eta,R,Etable,Efull,PhysThickInmm,tracknum,PhysThickInXo,dEdx,dEdxtimesdx,dEdxtimesdxCum,EtableCum,EfullCum,PhysThickInXoCum,layerflag,layer
461,Epoxy,3607.35,2.35502,690.836,0.018672,0,0.07,0.0,2.215884e-04,0.268491,0.018794,292.468430,337.449618,0,24.468821,True,28.0
462,WCu,3608.80,2.35502,691.114,2.667120,0,1.45,0.0,2.830649e-01,1.811710,2.626980,295.095410,340.116738,0,24.751886,True,28.0
463,Copper,3614.90,2.35502,692.282,9.003090,0,6.10,0.0,4.249124e-01,1.257088,7.668237,302.763647,349.119828,0,25.176798,True,28.0
464,WCu,3616.35,2.35502,692.560,2.665300,0,1.45,0.0,2.830649e-01,1.811710,2.626980,305.390626,351.785128,0,25.459863,True,28.0
465,Epoxy,3616.42,2.35502,692.573,0.018655,0,0.07,0.0,2.215884e-04,0.268491,0.018794,305.409421,351.803783,0,25.460085,True,28.0
466,Silicon,3616.54,2.35502,692.596,0.142709,0,0.12,0.0,1.281008e-03,0.387546,0.046505,305.455926,351.946492,0,25.461366,False,28.0
467,Silicon,3616.73,2.35502,692.632,0.225955,0,0.19,0.0,2.028263e-03,0.387546,0.073634,305.529560,352.172447,0,25.463394,False,28.0
468,Epoxy,3616.80,2.35502,692.645,0.018655,0,0.07,0.0,2.215884e-04,0.268491,0.018794,305.548354,352.191102,0,25.463615,True,29.0
469,Kapton,3616.90,2.35502,692.665,0.023141,0,0.10,0.0,2.737409e-04,0.216146,0.021615,305.569969,352.214243,0,25.463889,True,29.0
470,Epoxy,3616.97,2.35502,692.678,0.018654,0,0.07,0.0,2.215884e-04,0.268491,0.018794,305.588763,352.232897,0,25.464111,True,29.0


In [23]:
#matZ[matZ["Z"]>0]
#matZ[matZ["PhysThickInmm"]> 2000.]
#matZ[matZ["tracknum"] == 2 ]
#matZ[ (matZ["tracknum"] == 20) & (matZ["layer"] > 10) ]
#matZ[450:500]
#For the D28 cumulative material in front of layers. 
newmatZ = matZ[ (matZ['tracknum'] == 2.0) & (matZ['layerflag'] == True) & (matZ['layerflag'].shift(-1) == False)]
newmatZ[['layer', 'PhysThickInXoCum']]
newmatZ[['layer', 'PhysThickInXoCum']].round(3).to_csv(r'/Users/apsallid/D41.cumulative.xo', header=None, index=None, sep=' ', mode='a')

In [23]:
# The dEdx weights calculation doesn't include the sensitive material. 
matZ = matZ.drop(matZ[ (matZ.Mat == "Silicon") | (matZ.Mat == "H_Scintillator") ].index)

In [24]:
# We will add a column with the dedx weights. First dedxtimesdx sum per layer 
# This is with the theoretical dedx
mdEdxtimesdxsumperlayer = matZ.groupby(['tracknum','layer'])["dEdxtimesdx"].sum()
# And now with the detailed simulation
mdEdxtimesdxsumperlayer_detailedtable = matZ.groupby(['tracknum','layer'])["Etable"].sum()
mdEdxtimesdxsumperlayer_detailedfull = matZ.groupby(['tracknum','layer'])["Efull"].sum()

#type(mdEdxtimesdxsumperlayer)
mdEdxtimesdxsumperlayer = mdEdxtimesdxsumperlayer.to_frame().reset_index()
mdEdxtimesdxsumperlayer_detailedtable = mdEdxtimesdxsumperlayer_detailedtable.to_frame().reset_index()
mdEdxtimesdxsumperlayer_detailedfull = mdEdxtimesdxsumperlayer_detailedfull.to_frame().reset_index()

#mdEdxtimesdxsumperlayer
#type(mdEdxtimesdxsumperlayer)

In [25]:
#Let's put also the accumulated energy loss
mdEdxtimesdxsumperlayer["dEdxtimesdxCum"] = mdEdxtimesdxsumperlayer.groupby('tracknum')["dEdxtimesdx"].cumsum()
mdEdxtimesdxsumperlayer_detailedtable["EtableCum"] = mdEdxtimesdxsumperlayer_detailedtable.groupby('tracknum')["Etable"].cumsum()
mdEdxtimesdxsumperlayer_detailedfull["EfullCum"] = mdEdxtimesdxsumperlayer_detailedfull.groupby('tracknum')["Efull"].cumsum()

In [26]:
#Final weights
mdEdxtimesdxsumperlayer["dedxweights"] = (mdEdxtimesdxsumperlayer["dEdxtimesdx"] + mdEdxtimesdxsumperlayer["dEdxtimesdx"].shift(-1))/2
mdEdxtimesdxsumperlayer_detailedtable["dedxweights_detailedsimulationtable"] = (mdEdxtimesdxsumperlayer_detailedtable["Etable"] + mdEdxtimesdxsumperlayer_detailedtable["Etable"].shift(-1))/2
mdEdxtimesdxsumperlayer_detailedfull["dedxweights_detailedsimulationfull"] = (mdEdxtimesdxsumperlayer_detailedfull["Efull"] + mdEdxtimesdxsumperlayer_detailedfull["Efull"].shift(-1))/2

In [27]:
#Hack for the last layer
mdEdxtimesdxsumperlayer.loc[mdEdxtimesdxsumperlayer["layer"] == 50.0, "dedxweights"] = 86.929520
mdEdxtimesdxsumperlayer_detailedtable.loc[mdEdxtimesdxsumperlayer_detailedtable["layer"] == 52.0, "dedxweights_detailedsimulationtable"] = 91.800143
mdEdxtimesdxsumperlayer_detailedfull.loc[mdEdxtimesdxsumperlayer_detailedfull["layer"] == 52.0, "dedxweights_detailedsimulationfull"] = 98.971647

#Drop duplicates not needed columns
mdEdxtimesdxsumperlayer_detailedtable = mdEdxtimesdxsumperlayer_detailedtable.drop(['tracknum', 'layer'], axis=1)
mdEdxtimesdxsumperlayer_detailedfull = mdEdxtimesdxsumperlayer_detailedfull.drop(['tracknum', 'layer'], axis=1)

#mdEdxtimesdxsumperlayer[ mdEdxtimesdxsumperlayer['tracknum'] == 6.0  ]
#mdEdxtimesdxsumperlayer_detailed[ mdEdxtimesdxsumperlayer_detailed['tracknum'] == 6.0  ]
mdEdxtimesdxsumperlayer = pd.concat([mdEdxtimesdxsumperlayer, mdEdxtimesdxsumperlayer_detailedtable, mdEdxtimesdxsumperlayer_detailedfull], axis=1)
mdEdxtimesdxsumperlayer = mdEdxtimesdxsumperlayer.dropna()

#Adding two columns dedxtable/dedxtheory and dedxfull/dedxtheory
mdEdxtimesdxsumperlayer["EtableoverEtheory"] = mdEdxtimesdxsumperlayer["Etable"]/mdEdxtimesdxsumperlayer["dEdxtimesdx"]
mdEdxtimesdxsumperlayer["EfulloverEtheory"] = mdEdxtimesdxsumperlayer["Efull"]/mdEdxtimesdxsumperlayer["dEdxtimesdx"]
mdEdxtimesdxsumperlayer[ mdEdxtimesdxsumperlayer['tracknum'] == 2.0  ][:-1].dropna()

,tracknum,layer,dEdxtimesdx,dEdxtimesdxCum,dedxweights,Etable,EtableCum,dedxweights_detailedsimulationtable,Efull,EfullCum,dedxweights_detailedsimulationfull,EtableoverEtheory,EfulloverEtheory
101,2.0,1.0,4.829298,4.829298,8.894541,5.450458,5.450458,9.969817,0,0,0.0,1.128623,0.0
102,2.0,2.0,12.959785,17.789083,10.937907,14.489176,19.939635,12.230183,0,0,0.0,1.118011,0.0
103,2.0,3.0,8.916030,26.705112,10.937907,9.971190,29.910825,12.207423,0,0,0.0,1.118344,0.0
104,2.0,4.0,12.959785,39.664897,10.937907,14.443655,44.354480,12.194097,0,0,0.0,1.114498,0.0
105,2.0,5.0,8.916030,48.580927,10.937907,9.944539,54.299020,12.171758,0,0,0.0,1.115355,0.0
106,2.0,6.0,12.959785,61.540711,10.937907,14.398976,68.697995,12.158694,0,0,0.0,1.111051,0.0
107,2.0,7.0,8.916030,70.456741,10.937907,9.918413,78.616408,12.136771,0,0,0.0,1.112425,0.0
108,2.0,8.0,12.959785,83.416525,10.937907,14.355129,92.971537,12.123950,0,0,0.0,1.107667,0.0
109,2.0,9.0,8.916030,92.332555,10.937907,9.892771,102.864307,12.102432,0,0,0.0,1.109549,0.0
110,2.0,10.0,12.959785,105.292340,10.932882,14.312094,117.176401,12.089853,0,0,0.0,1.104347,0.0


In [28]:
mdEdxtimesdxsumperlayer[["layer","EtableoverEtheory","EfulloverEtheory"]][ mdEdxtimesdxsumperlayer['tracknum'] == 2.0  ][:-1] 

,layer,EtableoverEtheory,EfulloverEtheory
101,1.0,1.128623,0.0
102,2.0,1.118011,0.0
103,3.0,1.118344,0.0
104,4.0,1.114498,0.0
105,5.0,1.115355,0.0
106,6.0,1.111051,0.0
107,7.0,1.112425,0.0
108,8.0,1.107667,0.0
109,9.0,1.109549,0.0
110,10.0,1.104347,0.0


In [29]:
mdEdxtimesdxsumperlayer[["layer","dedxweights","dedxweights_detailedsimulationtable","dedxweights_detailedsimulationfull"]][ mdEdxtimesdxsumperlayer['tracknum'] == 2.0  ][:-1] 

,layer,dedxweights,dedxweights_detailedsimulationtable,dedxweights_detailedsimulationfull
101,1.0,8.894541,9.969817,0.0
102,2.0,10.937907,12.230183,0.0
103,3.0,10.937907,12.207423,0.0
104,4.0,10.937907,12.194097,0.0
105,5.0,10.937907,12.171758,0.0
106,6.0,10.937907,12.158694,0.0
107,7.0,10.937907,12.136771,0.0
108,8.0,10.937907,12.123950,0.0
109,9.0,10.937907,12.102432,0.0
110,10.0,10.932882,12.089853,0.0


In [30]:
mdEdxtimesdxsumperlayer[["layer","dEdxtimesdxCum","EtableCum","EfullCum"]][ mdEdxtimesdxsumperlayer['tracknum'] == 2.0  ][:-1] 

,layer,dEdxtimesdxCum,EtableCum,EfullCum
101,1.0,4.829298,5.450458,0
102,2.0,17.789083,19.939635,0
103,3.0,26.705112,29.910825,0
104,4.0,39.664897,44.354480,0
105,5.0,48.580927,54.299020,0
106,6.0,61.540711,68.697995,0
107,7.0,70.456741,78.616408,0
108,8.0,83.416525,92.971537,0
109,9.0,92.332555,102.864307,0
110,10.0,105.292340,117.176401,0
